In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

import pickle

transactions = pd.read_csv('../transactions.csv', index_col='transactionId')

In [2]:
transactions.head(10)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
transactionId,,,,,,,,,,
0,1,3,9839.64,1.123101e+10,170136.00,160296.36,2.197979e+10,0.0,0.00,0
1,1,3,1864.28,1.166654e+10,21249.00,19384.72,2.204428e+10,0.0,0.00,0
2,1,4,181.00,1.130549e+10,181.00,0.00,1.553264e+09,0.0,0.00,1
3,1,1,181.00,1.840084e+09,181.00,0.00,1.389970e+08,21182.0,0.00,1
4,1,3,11668.14,1.204854e+10,41554.00,29885.86,2.123070e+10,0.0,0.00,0
5,1,3,7817.71,1.900456e+08,53860.00,46042.29,2.573487e+09,0.0,0.00,0
6,1,3,7107.77,1.154989e+09,183195.00,176087.23,2.408069e+09,0.0,0.00,0
7,1,3,7861.64,1.191285e+10,176087.23,168225.59,2.633326e+09,0.0,0.00,0
8,1,3,4024.36,1.126501e+10,2671.00,0.00,2.117693e+10,0.0,0.00,0


In [3]:
X = transactions.drop(columns=['isFraud'])
y = transactions['isFraud']
smote = SMOTE(random_state=42)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [4]:
param_grid = {
    'n_neighbors': np.arange(1, 11),  # Range of neighbors to try
    'weights': ['uniform', 'distance'],  # Weighting options
    'p': [1, 2]  # Power parameter for the Minkowski distance
}

knn_classifier = KNeighborsClassifier()

grid_search = GridSearchCV(knn_classifier, param_grid)
grid_search.fit(X_train, y_train)

# Display the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Get the best model from the grid search
best_knn_model = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred = best_knn_model.predict(X_test)

# Evaluate the performance of the best model
accuracy = np.mean(y_pred == y_test)
print("Accuracy of the Best Model:", accuracy)

In [9]:
pickle.dump(best_knn_model, open('./KNN.pkl', 'wb'))